In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════╗
# ║  CELL 1: LA RENT RELIEF - COLAB ENVIRONMENT SETUP                                    ║
# ║  🔥 FLAN-T5 = ORCHESTRATOR | PYTHON = EXECUTOR/VALIDATOR║
# ╚══════════════════════════════════════════════════════════════════════════════════════╝

import subprocess
import sys

print("=" * 90)
print("🔧 INSTALLING DEPENDENCIES")
print("=" * 90)

packages = [
    'transformers>=4.30.0', 'torch>=2.0.0', 'accelerate>=0.20.0',
    'sentencepiece', 'rapidfuzz', 'bitsandbytes',
    'requests', 'aiohttp'
]
for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])
    print(f"   ✅ {pkg}")

import pandas as pd
import numpy as np
import os
import re
import json
import gc
import hashlib
import requests
from datetime import datetime
from google.colab import drive
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Mount Google Drive
print("\n📁 Mounting Google Drive...")
drive.mount('/content/drive', force_remount=False)

# Create outputs folder
os.makedirs('/content/outputs', exist_ok=True)

print("\n" + "=" * 90)
print("🤖 LOADING FLAN-T5 MODEL (THE ORCHESTRATOR)")
print("=" * 90)

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE.upper()}")
if DEVICE == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

MODEL_NAME = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if DEVICE == 'cuda' else torch.float32,
    device_map='auto' if DEVICE == 'cuda' else None
)
if DEVICE == 'cpu':
    model = model.to(DEVICE)
model.eval()
print(f"✅ FLAN-T5 loaded on {DEVICE.upper()}")

def flan_t5_generate(prompt, max_length=64):
    """LLM ORCHESTRATOR: Makes intelligent decisions"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_beams=1, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def llm_classify(text, options):
    """LLM decides classification"""
    prompt = f"Classify '{text}' into one of: {', '.join(options)}. Answer with just the category."
    return flan_t5_generate(prompt, max_length=16)

def llm_extract(text, field):
    """LLM extracts specific field from text"""
    prompt = f"Extract the {field} from: '{text}'. Return only the {field} or'NONE' if not found."
    return flan_t5_generate(prompt, max_length=32)

def llm_validate_match(name1, name2):
    """LLM validates if two names are the same entity"""
    prompt = f"Are '{name1}' and '{name2}' the same business entity? Answer yes or no."
    result = flan_t5_generate(prompt, max_length=8).lower()
    return 'yes' in result

def llm_score_prospect(data_dict):
    """LLM generates prospect score rationale"""
    prompt = f"Rate urgency 1-100 for landlord with ${data_dict.get('delinquent_amount', 0):,.0f} tax debt and ${data_dict.get('ppp_loan_amount', 0):,.0f} PPP loan. Just the number."
    try:
        return int(flan_t5_generate(prompt, max_length=8))
    except:
        return 50

# Test
test = flan_t5_generate("Classify: SMITH PROPERTIES LLC. Options: LLC, INC, CORP, INDIVIDUAL")
print(f"🧪 Test:'SMITH PROPERTIES LLC' → '{test}'")

print("\n✅ CELL 1 COMPLETE - LLM Orchestrator Ready")